In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Code to accompany the paper "Constructions in combinatorics via neural networks and LP solvers" by A Z Wagner
# Code for conjecture 2.1, without the use of numba 
#
# Please keep in mind that I am far from being an expert in reinforcement learning. 
# If you know what you are doing, you might be better off writing your own code.
#
# This code works on tensorflow version 1.14.0 and python version 3.6.3
# It mysteriously breaks on other versions of python.
# For later versions of tensorflow there seems to be a massive overhead in the predict function for some reason, and/or it produces mysterious errors.
# Debugging these was way above my skill level.
# If the code doesn't work, make sure you are using these versions of tf and python.
# I used keras version 2.3.1, not sure if this is important, but I recommend this just to be safe.

import networkx as nx #for various graph parameters, such as eigenvalues, macthing number, etc
import random
import numpy as np
import copy
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.models import load_model
from statistics import mean
from math import sqrt
from numpy.random import choice
import pickle
import time
import math
import matplotlib.pyplot as plt

In [3]:
import randstab as rs
import utils as utils
import kron_vec_product as kron_fast

In [4]:
from stabilizer_search.search.brute_force import *
from stabilizer_search.mat import X, T
from stabilizer_search.mat import tensor

In [5]:
I = np.eye(2);
X = np.array([[0,1],[1,0]]);
Y = np.array([[0,-1j],[1j,0]]);
Z = np.array([[1,0],[0,-1]]);
pauli = {0: I, 1: X, 2: Y, 3: Z};
coefficient = {0: 1, 1: -1, 2: 1j, 3: -1j}

In [6]:
def update(state, pauli_number, n):
    assert 0 <= pauli_number <= 4**(n+1)-1
    implicit_pauli_string = np.zeros(n+1)
    filler = utils.numberToBase(pauli_number,4)
    implicit_pauli_string[-len(filler):] = filler
    explicit_pauli_string = [pauli[implicit_pauli_string[i]] for i in range(n)]
#     state_new = state + kron_fast.kron_vec_prod(explicit_pauli_string, state)
    state_new = state + coefficient[implicit_pauli_string[n]]*kron_fast.kron_vec_prod(explicit_pauli_string, state)
    if np.linalg.norm(state_new) == 0:
#         print("NORM is zero" - note norm can be zero from e.g. (1+X)|-> = 0)
        return state  
#     print("NORM is not zero" - result of experiment: usually norm isn't zero, not a big problem)
    assert np.linalg.norm(state) != 0
    state_new = state_new/np.linalg.norm(state_new)
    return state_new

In [7]:
n_qubits = 3
chi = 3
n_paulis = 4**(n_qubits+1)

In [8]:
H = [[np.cos(np.pi/8)],[np.sin(np.pi/8)]]
T_state = np.array([[1],[np.exp(1j*np.pi/4)]])/np.sqrt(2)
T_perp_state = np.matrix(Z)*T_state
target_magic_state = tensor(*([H]*n_qubits))
# target_magic_state = (tensor(*([T_state]*n_qubits)) + tensor(*([T_perp_state]*n_qubits)))/np.sqrt(2)

In [9]:
Z0 = [[1],[0]]
# tranpose needed to input base_stabilizer_states into update function
base_stabilizer_states = [tensor(*([Z0]*n_qubits)).transpose()]*chi
# base_stabilizer_states = [np.array([rs.random_stabilizer_state(n_qubits)]) for i in range(chi)]

In [10]:
# Daochen: would be easier if could enforce the use of combinations
# MYN = int(2**n)  #The length of the word we are generating. Here we are generating a Boolean function on n bits, so we create a 0-1 word of length 2^n

MYN = int(chi)

# LEARNING_RATE = 0.0001 #Increase this to make convergence faster, decrease if the algorithm gets stuck in local optima too often.
LEARNING_RATE = 0.0001
n_sessions = 500 #number of new sessions per iteration
# default 93, 94 respectively
percentile = 96 #top 100-X percentiled we are learning from
super_percentile = 97 #top 100-X percentile that survives to next iteration

# These are hyperparameters
FIRST_LAYER_NEURONS = 128 #Number of neurons in the hidden layers.
SECOND_LAYER_NEURONS = 64
# THIRD_LAYER_NEURONS = 4
THIRD_LAYER_NEURONS = 32

# n_actions = 2
# Daochen: note that this parameter is not actually used anywhere.
n_actions = n_paulis
#The size of the alphabet. In this file we will assume this is 2. There are a few things we need to change when the alphabet size is larger,
#such as one-hot encoding the input, and using categorical_crossentropy as a loss function.

observation_space = 2*MYN 

# Leave this at 2*MYN. The input vector will have size 2*MYN, 
# where the first MYN letters encode our partial word (with zeros on
# the positions we haven't considered yet), and the next MYN bits one-hot encode which letter we are considering now.
# So e.g. [0,1,0,0,   0,0,1,0] means we have the partial word 01 and we are considering the third letter now.
# Is there a better way to format the input to make it easier for the neural network to understand things?

# Daochen: why should len_game have anything to do with MYN
len_game = MYN 
state_dim = (observation_space,)

INF = 1000000

#Model structure: a sequential network with three hidden layers, sigmoid activation in the output.
#I usually used relu activation in the hidden layers but play around to see what activation function and what optimizer works best.
#It is important that the loss is binary cross-entropy if alphabet size is 2.

model = Sequential()
model.add(Dense(FIRST_LAYER_NEURONS,  activation="relu"))
model.add(Dense(SECOND_LAYER_NEURONS, activation="relu"))
model.add(Dense(THIRD_LAYER_NEURONS, activation="relu"))
model.add(Dense(n_paulis, activation="softmax"))
model.build((None, observation_space))
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate = LEARNING_RATE)) #Adam optimizer also works well, with lower learning rate
weights_init = model.get_weights()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8448      
Total params: 19,680
Trainable params: 19,680
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
def calcScore(state):
    """
    Calculates the reward for a given word. 
    This function is very slow, it can be massively sped up with numba -- but numba doesn't support networkx yet, which is very convenient to use here
    :param state: the first MYN letters of this param are the word that the neural network has constructed.

    :returns: the reward (a real number). Higher is better, the network will try to maximize this.
    """

    f = state[:MYN];
    candidate_stabilizer_basis = [update(base_stabilizer_states[i], f[i], n_qubits).transpose() for i in range(MYN)]
    projector = ortho_projector(candidate_stabilizer_basis)
    projection = np.linalg.norm(projector*target_magic_state, 2)
#     projection = 1
    
    score = projection
    target = score
    
    if np.allclose(score, 1):
        print('You found a stabilizer decomposition with (n_qubits,chi) = ', [n_qubits,chi])
        print('The set of stabilizers is: ', f)
        return -1, -1
    return target, score

In [12]:
####No need to change anything below here. 
# Daochen: the agent argument will be the "model"
def generate_session(agent, n_sessions, verbose = 1):
    """
    Play n_session games using agent neural network.
    Terminate when games finish 

    Code inspired by https://github.com/yandexdataschool/Practical_RL/blob/master/week01_intro/deep_crossentropy_method.ipynb
    """
    states =  np.zeros([n_sessions, observation_space, len_game], dtype=int)
    actions = np.zeros([n_sessions, len_game], dtype = int)
    state_next = np.zeros([n_sessions,observation_space], dtype = int)
    prob = np.zeros(n_sessions)
    states[:,MYN,0] = 1
    step = 0
    total_target = np.zeros([n_sessions])
#     total_target = np.zeros([n_sessions], dtype=complex)
    total_score = np.zeros([n_sessions])
    recordsess_time = 0
    play_time = 0
    scorecalc_time = 0
    pred_time = 0
    while (True):
        step += 1
        tic = time.time()
        prob = agent.predict(states[:,:,step-1], batch_size = n_sessions) 
        pred_time += time.time()-tic

        for i in range(n_sessions):
#             Daochen: important action selection step
            action = choice(n_paulis, p=prob[i])
#             print('probability vector: ', prob[i],' and action chosen: ', action)
            actions[i][step-1] = action
            tic = time.time()
            state_next[i] = states[i,:,step-1]
            play_time += time.time()-tic
#             if (action > 0):
#                 state_next[i][step-1] = action
            state_next[i][step-1] = action
            state_next[i][MYN + step-1] = 0
            if (step < MYN):
                state_next[i][MYN + step] = 1
#                 Daochen: terminal equals whether step equals MYN: I suppose meaning that an entire state has been generated
            terminal = step == MYN
            tic = time.time()
            if terminal:
#                 print('state_next[i]', state_next[i])
                total_target[i], total_score[i] = calcScore(state_next[i])
                if total_target[i] == -1:
                    return -1
#                 print("total_score", total_score[i])
            scorecalc_time += time.time()-tic
            tic = time.time()
            if not terminal:
                states[i,:,step] = state_next[i]
            recordsess_time += time.time()-tic
        if terminal:
            break
    #If you want, print out how much time each step has taken. This is useful to find the bottleneck in the program.		
    if (verbose):
        print("Predict: "+str(pred_time)+", play: " + str(play_time) +", scorecalc: " + str(scorecalc_time) +", recordsess: " + str(recordsess_time))
    return states, actions, total_score, total_target

def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    This function was mostly taken from https://github.com/yandexdataschool/Practical_RL/blob/master/week01_intro/deep_crossentropy_method.ipynb
    If this function is the bottleneck, it can easily be sped up using numba
    """
    counter = n_sessions * (100.0 - percentile) / 100.0
    reward_threshold = np.percentile(rewards_batch,percentile)

    elite_states = []
    elite_actions = []
    elite_rewards = []
    for i in range(len(states_batch)):
        if rewards_batch[i] >= reward_threshold-0.0000001:
            if (counter > 0) or (rewards_batch[i] >= reward_threshold+0.0000001):
                for item in states_batch[i]:
                    elite_states.append(item.tolist())
                for item in actions_batch[i]:
                    elite_actions.append(item)
            counter -= 1
    elite_states = np.array(elite_states, dtype = int)
    elite_actions = np.array(elite_actions, dtype = int)
    return elite_states, elite_actions

def select_super_sessions(states_batch, actions_batch, rewards_batch, targets_batch, percentile=90):
    """
    Select all the sessions that will survive to the next generation
    Similar to select_elites function
    If this function is the bottleneck, it can easily be sped up using numba
    """
    counter = n_sessions * (100.0 - percentile) / 100.0
    reward_threshold = np.percentile(rewards_batch,percentile)

    super_states = []
    super_actions = []
    super_rewards = []
    super_targets = []
    for i in range(len(states_batch)):
        if rewards_batch[i] >= reward_threshold-0.0000001:
            if (counter > 0) or (rewards_batch[i] >= reward_threshold+0.0000001):
                super_states.append(states_batch[i])
                super_actions.append(actions_batch[i])
                super_rewards.append(rewards_batch[i])
                super_targets.append(targets_batch[i])
                counter -= 1
    super_states = np.array(super_states, dtype = int)
    super_actions = np.array(super_actions, dtype = int)
    super_rewards = np.array(super_rewards)
    super_targets = np.array(super_targets)
    return super_states, super_actions, super_rewards, super_targets

In [13]:
super_states =  np.empty((0,len_game,observation_space), dtype = int)
super_actions = np.array([], dtype = int)
super_rewards = np.array([])
super_targets= np.array([])
sessgen_time = 0
fit_time = 0
score_time = 0

myRand = random.randint(0,1000) #used in the filename

In [14]:
for i in range(1000000): #1000000 generations should be plenty
    #generate new sessions
    #performance can be improved with joblib
    tic = time.time()
#     sessions = states, actions, total_score, total_target
    sessions = generate_session(model,n_sessions,0) #change 0 to 1 to print out how much time each step in generate_session takes 
    if sessions == -1:
        break
    sessgen_time = time.time()-tic
    tic = time.time()

    states_batch = np.array(sessions[0], dtype = int)
    actions_batch = np.array(sessions[1], dtype = int)
    rewards_batch = np.array(sessions[2])
    targets_batch = np.array(sessions[3])
    
    if i%2 == 0 and i > 1:
        action = actions_batch[0]
#         print('Action:', action)
        print('Updated base_stabilizer_states at iteration: ', i)
        base_stabilizer_states = [update(base_stabilizer_states[i], action[i], n_qubits) for i in range(MYN)]
        model.set_weights(weights_init)
#         print('debug', base_stabilizer_states)
    
    states_batch = np.transpose(states_batch,axes=[0,2,1])
    states_batch = np.append(states_batch,super_states,axis=0)

    if i>0:
        actions_batch = np.append(actions_batch,np.array(super_actions),axis=0)	
    
    rewards_batch = np.append(rewards_batch,super_rewards)
    targets_batch = np.append(targets_batch,super_targets)

    randomcomp_time = time.time()-tic 
    tic = time.time()

    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile=percentile) #pick the sessions to learn from
    select1_time = time.time()-tic

    tic = time.time()
    super_sessions = select_super_sessions(states_batch, actions_batch, rewards_batch, targets_batch, percentile=super_percentile) #pick the sessions to survive
    select2_time = time.time()-tic

    tic = time.time()
    super_sessions = [(super_sessions[0][i], super_sessions[1][i], super_sessions[2][i], super_sessions[3][i]) for i in range(len(super_sessions[2]))]
    super_sessions.sort(key=lambda super_sessions: super_sessions[2],reverse=True)
    select3_time = time.time()-tic

    tic = time.time()
    model.fit(elite_states, elite_actions) #learn from the elite sessions
#     print('elite_states',elite_states)
#     print('elite_actions',elite_actions)
    fit_time = time.time()-tic

    tic = time.time()

    super_states = [super_sessions[i][0] for i in range(len(super_sessions))]
    super_actions = [super_sessions[i][1] for i in range(len(super_sessions))]
    super_rewards = [super_sessions[i][2] for i in range(len(super_sessions))]
    super_targets = [super_sessions[i][3] for i in range(len(super_sessions))]
    
#     print(super_states)

    rewards_batch.sort()
#     Daochen: why is it -100?
    mean_all_reward = np.mean(rewards_batch[-100:])
    mean_best_reward = np.mean(super_rewards)

    score_time = time.time()-tic

    print("\n" + str(i) +  ". Best individuals (reward): " + str(np.flip(np.sort(super_rewards))))
#     Daochen: note that sometimes it makes sense to add/remove np.flip below
#     print("\n" +  ". Best individuals (target): " + str(np.flip(np.sort(super_targets))))

    #uncomment below line to print out how much time each step in this loop takes. 
#     print("Mean reward: " + str(mean_all_reward) + "\nSessgen: " + str(sessgen_time) + ", other: " + str(randomcomp_time) + ", select1: " + str(select1_time) + ", select2: " + str(select2_time) + ", select3: " + str(select3_time) +  ", fit: " + str(fit_time) + ", score: " + str(score_time))
#     if (i%25 == 1): #Write all important info to files every 20 iterations
#         with open('best_species_pickle_'+str(myRand)+'.txt', 'wb') as fp:
#             pickle.dump(super_actions, fp)
#         with open('best_species_txt_'+str(myRand)+'.txt', 'w') as f:
#             for item in super_actions:
#                 f.write(str(item))
#                 f.write("\n")
#         with open('best_species_rewards_'+str(myRand)+'.txt', 'w') as f:
#             for item in super_rewards:
#                 f.write(str(item))
#                 f.write("\n")
#         with open('best_100_rewards_'+str(myRand)+'.txt', 'a') as f:
#             f.write(str(mean_all_reward)+"\n")
#         with open('best_elite_rewards_'+str(myRand)+'.txt', 'a') as f:
#             f.write(str(mean_best_reward)+"\n")
#     if (i%200==2): # To create a timeline, like in Figure 3
#         with open('best_species_timeline_txt_'+str(myRand)+'.txt', 'a') as f:
#             f.write(str(super_actions[0]))
#             f.write("\n")

2/2 [==============================] - 0s 4ms/step - loss: 4.2767

0. Best individuals (reward): [0.91891266 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188  0.9139188  0.9139188 ]
2/2 [==============================] - 0s 4ms/step - loss: 3.9840

1. Best individuals (reward): [0.92387953 0.91891266 0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188 ]
Updated base_stabilizer_states at iteration:  2
3/3 [==============================] - 0s 4ms/step - loss: 4.6670

2. Best individuals (reward): [0.92387953 0.91891266 0.91891266 0.91891266 0.91891266 0.91891266
 0.91477754 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188  0.9139188  0.9139188  0.9139188  0.9139188  0.9139188
 0.9139188 ]
2/2 [==============================] - 0s 3ms/step - loss: 4.5852

3. Best individuals (rewar

Updated base_stabilizer_states at iteration:  28
2/2 [==============================] - 0s 5ms/step - loss: 4.3256

28. Best individuals (reward): [0.94348558 0.93892832 0.93727511 0.93727511 0.93727511 0.93569246
 0.93535674 0.92827222 0.92774041 0.92523836 0.92503949 0.92455919
 0.92455919 0.92455919 0.92421943 0.92416279]
3/3 [==============================] - 0s 5ms/step - loss: 3.5475

29. Best individuals (reward): [0.94348558 0.93892832 0.93727511 0.93727511 0.93727511 0.93569246
 0.93535674 0.92827222 0.92774041 0.92523836 0.92503949 0.92455919
 0.92455919 0.92455919 0.92421943 0.92416279]
Updated base_stabilizer_states at iteration:  30
2/2 [==============================] - 0s 3ms/step - loss: 4.1365

30. Best individuals (reward): [0.94348558 0.93892832 0.93727511 0.93727511 0.93727511 0.93569246
 0.93535674 0.92827222 0.92774041 0.92523836 0.92503949 0.92455919
 0.92455919 0.92455919 0.92421943 0.92416279]
2/2 [==============================] - 0s 3ms/step - loss: 4.1131

3

3/3 [==============================] - 0s 3ms/step - loss: 3.0821

55. Best individuals (reward): [0.9543017  0.9543017  0.95073362 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378 0.94348558
 0.94022242 0.93892832 0.93727511 0.91822477 0.91822477 0.91822477
 0.91822477]
Updated base_stabilizer_states at iteration:  56
2/2 [==============================] - 0s 4ms/step - loss: 3.2775

56. Best individuals (reward): [0.9543017  0.9543017  0.95073362 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378 0.94348558
 0.94022242 0.93892832 0.93727511]
2/2 [==============================] - 0s 4ms/step - loss: 3.8140

57. Best individuals (reward): [0.9543017  0.9543017  0.95073362 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378 0.94348558
 0.94022242 0.93892832 0.93727511 0.80693775]
Updated base_stabilizer_states at iteration:  58
3/3 [==============================] - 0s 3ms

2/2 [==============================] - 0s 3ms/step - loss: 3.8873

82. Best individuals (reward): [0.9543017  0.9543017  0.95400998 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378
 0.94348558 0.94022242 0.9396894  0.9396894 ]
2/2 [==============================] - 0s 3ms/step - loss: 3.7382

83. Best individuals (reward): [0.9543017  0.9543017  0.95400998 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378
 0.94348558 0.94022242 0.9396894  0.9396894 ]
Updated base_stabilizer_states at iteration:  84
2/2 [==============================] - 0s 7ms/step - loss: 3.8645

84. Best individuals (reward): [0.9543017  0.9543017  0.95400998 0.95073362 0.95073362 0.95073362
 0.95073362 0.95073362 0.95073362 0.95073362 0.94671378 0.94671378
 0.94348558 0.94022242 0.9396894  0.9396894 ]
2/2 [==============================] - 0s 13ms/step - loss: 3.6919

85. Best individuals (reward): [0.9543017  0.9543

2/2 [==============================] - 0s 3ms/step - loss: 3.5575

109. Best individuals (reward): [0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.95473653
 0.9543017  0.9543017  0.95400998 0.95313738 0.95313738 0.95313738
 0.95203835 0.95073362 0.95073362]
Updated base_stabilizer_states at iteration:  110
4/4 [==============================] - 0s 4ms/step - loss: 3.4271

110. Best individuals (reward): [0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.95473653
 0.9543017  0.9543017  0.95400998 0.95313738 0.95313738 0.95313738
 0.95203835 0.95073362 0.95073362 0.93609864]
3/3 [==============================] - 0s 3ms/step - loss: 3.4617

111. Best individuals (reward): [0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.95710678
 0.95473653 0.9543017  0.9543017  0.95400998 0.95313738 0.95313738
 0.95313738 0.95203835 0.95073362]
Updated base_stabilizer_states at iteration:  112
3/3 [==============================] - 0s 2ms/step - loss: 3.7841

112. Best individu

...

2/2 [==============================] - 0s 2ms/step - loss: 3.4306

459. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
Updated base_stabilizer_states at iteration:  460
2/2 [==============================] - 0s 2ms/step - loss: 3.6230

460. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
2/2 [==============================] - 0s 3ms/step - loss: 3.7213

461. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
Updated base_stabilizer_states at iteration:  462
3/3 [==============================] - 0s 2ms/step - loss: 3.69

2/2 [==============================] - 0s 3ms/step - loss: 3.8371

486. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
3/3 [==============================] - 0s 2ms/step - loss: 3.4597

487. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
Updated base_stabilizer_states at iteration:  488
3/3 [==============================] - 0s 2ms/step - loss: 3.4426

488. Best individuals (reward): [0.97289167 0.9690101  0.96786573 0.96786573 0.96738813 0.96738813
 0.96425244 0.96160431 0.96160431 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.95948054]
2/2 [==============================] - 0s 2ms/step - loss: 3.6668

489. Best individuals (reward): [0.97289167 0.

2/2 [==============================] - 0s 2ms/step - loss: 3.7180

513. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.8851389 ]
Updated base_stabilizer_states at iteration:  514
2/2 [==============================] - 0s 2ms/step - loss: 3.9059

514. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.89242089]
3/3 [==============================] - 0s 3ms/step - loss: 3.4996

515. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.89242089]
Updated base_stabilizer_states at iteration:  516
3/3 [==============================] - 0s 2ms/step - loss: 3.55

2/2 [==============================] - 0s 2ms/step - loss: 3.8116

540. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.94749542]
3/3 [==============================] - 0s 2ms/step - loss: 3.2493

541. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.94749542]
Updated base_stabilizer_states at iteration:  542
2/2 [==============================] - 0s 2ms/step - loss: 3.4533

542. Best individuals (reward): [0.97289167 0.97289167 0.9690101  0.96786573 0.96786573 0.96738813
 0.96738813 0.96738813 0.96425244 0.96160431 0.96160431 0.96160431
 0.96160431 0.96160431 0.96160431 0.94749542]
2/2 [==============================] - 0s 3ms/step - loss: 3.7234

543. Best individuals (reward): [0.97289167 0.

KeyboardInterrupt: 